In [ ]:
%load_ext autoreload
%autoreload 2

## API endpoints:


I. Endpoints for fetching rules

- Give me all the rules
- Give me rules for the datatypes X,Y,Z
- Give me rules applicable to the datatypes X,Y,Z given input filters

They return:
- a list of rules with their meta-data: id/name/desc (premise/query ?)

---------------------

II. Endpoinds for applying rules

- Apply rule A given input filters
- Apply rules A,B,C given input filters (?)

They return:
- a list of result resource IDs or full payloads (ids passed to ES views by atals?)


ISSUES TO RESOLVE:

- Convention for the filters: how atlas knows which params need to be specified for different rules
- Permission:
    - fetching rules: service account that can always access the project with rules.
    - applying rules: we need to get USER token from atlas and use it when applying the rules. handle access denied in some sessions.
- Service sessions for storing forge sessions for different users.
- Pagination of results.

In [ ]:
import getpass

from kgforge.core import KnowledgeGraphForge

from inference_tools.utils import fetch_rules, check_premises, apply_rule, get_rule_parameters

In [ ]:
ENDPOINT = "https://bbp.epfl.ch/nexus/v1"

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
RULES_ORG = "bbp"
RULES_PROJECT = "inference-rules"
RULES_VIEW = "https://bbp.epfl.ch/neurosciencegraph/data/rule-view"

In [ ]:
rules_forge = KnowledgeGraphForge(
    "../../configs/new-forge-config.yaml",
    endpoint=ENDPOINT,
    token=TOKEN, 
    bucket=f"{RULES_ORG}/{RULES_PROJECT}")

In [ ]:
# This could be replaced by any util kg-inference-api implements to handle user forge sessions
FORGE_SESSIONS = {}
def allocate_forge_session(org, project):
    if (org, project) not in FORGE_SESSIONS:
        session = KnowledgeGraphForge(
            "../../configs/new-forge-config.yaml",
            endpoint=ENDPOINT,
            token=TOKEN, 
            bucket=f"{org}/{project}")
        FORGE_SESSIONS[(org, project)] = session
    return FORGE_SESSIONS[(org, project)]

# Fetch rules

1. Fetch all the rules

In [ ]:
rules = fetch_rules(rules_forge, RULES_VIEW)
for r in rules:
    print("Name: ", r["name"])
    print("Desc: ", r["description"])
    print("Target resource type: ", r["targetResourceType"])
    params = get_rule_parameters(r)
    print("Input parameters: ")
    for name, payload in params.items():
        print("\t", name)
        print("\t\t", payload)
    print()

2. Fetch rules for a particular data type

In [ ]:
morph_rules = fetch_rules(rules_forge, RULES_VIEW, resource_types=["NeuronMorphology"])
for r in morph_rules:
#     print(r)
    print("Name: ", r["name"])
    print("Desc: ", r["description"])
    print("Target resource type: ", r["targetResourceType"])
    params = get_rule_parameters(r)
    print("Input parameters: ")
    for name, payload in params.items():
        print("\t", name)
        print("\t\t", payload)
    print()

# Check premises of rules (find rules applicable to input filters)

In [ ]:
input_filters = {
    "TargetResourceParameter": "https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/85a140b6-8495-4838-9e4f-e347b2a7838f"
}

If premise is satified given the input filters, the rule is applicable.

In [ ]:
for rule in morph_rules:
    print(rule["name"])
    print("\t-> Input filters:")
    print("\t\tPremise satisfied: ", check_premises(allocate_forge_session, rule, input_filters))

# Excecute inference

Rule application returns a collection of resources to display in the atlas

By default the premise is checked again when the rule is applied

In [ ]:
for rule in morph_rules:
    print(rule["name"])
    print("\t Results:")
    results = apply_rule(allocate_forge_session, rule, input_filters)
    for r in results:
        print("\t\t", r)
    print()

## Real atlas example

In [ ]:
brain_regions = [
    # non cortical
    "mba:549", "mba:856", "mba:864", "mba:239", "mba:958", "mba:1014",
    # cortical
    "mba:315", "mba:541", "mba:97"
]

In [ ]:
mtypes = ["http://uri.interlex.org/base/ilx_0381377", "http://uri.interlex.org/base/ilx_038"]

In [ ]:
rule = rules_forge.as_json(rules_forge.retrieve(
    "https://bbp.epfl.ch/neurosciencegraph/data/99b71e48-abd5-49e9-965e-a08534b5f0fa"))

In [ ]:
input_filters = {
    "BrainRegionQueryParameter": brain_regions,
    "MTypeQueryParameter": mtypes
}

In [ ]:
satisfies = check_premises(allocate_forge_session, rule, input_filters)
print(" premise satisfied? ", satisfies)
if satisfies:
    results = apply_rule(
        allocate_forge_session, rule, input_filters, premise_check=False)
    print(results)